In [ ]:
import pandas as pd

import pyteomics.mgf
from tqdm.notebook import tqdm

In [ ]:
infile = r"C:\git\msn_library\data\gnpslib\ALL_GNPS_NO_PROPOGATED.mgf"
df_file = r"C:\git\msn_library\data\final_tables\gnps_phase.tsv"

### Group all rows with the same content
This needs to move to the script that generates the tsv file in the first place

In [ ]:
filtered_df = pd.read_csv(df_file, sep='\t')
filtered_df = filtered_df.rename(columns={"clinical_phase": "clinical_phase_no_drugbank", "clinical_phase2": "clinical_phase"})
filtered_df = filtered_df[filtered_df["clinical_phase"]>0]
filtered_df["none"] = filtered_df.isnull().sum(axis=1)
filtered_df = filtered_df.sort_values(by="none", ascending=True).drop_duplicates(['gnps_libid']).sort_values(by="clinical_phase", ascending=False)

cols = [col for col in filtered_df.columns if col not in ["gnps_libid", "entries"]]
filtered_df = filtered_df.groupby(cols, dropna=False)['gnps_libid'].apply(', '.join).reset_index()
filtered_df["entries"] = [str(lib).count(",") + 1 for lib in filtered_df["gnps_libid"]]
filtered_df.sort_values(["clinical_phase", "inchi_key"], ascending=[False, True], inplace=True)

filtered_df.to_csv(r"C:\git\msn_library\data\final_tables\gnps_drugs_clinical_phase_greater0.tsv", sep="\t", index=False)

### Start creating mgf from library

In [ ]:
# read tsv
filtered_df = pd.read_csv(df_file, sep='\t')
filtered_df = filtered_df.rename(columns={"clinical_phase": "clinical_phase_no_drugbank", "clinical_phase2": "clinical_phase"})
filtered_df = filtered_df[filtered_df["clinical_phase"]>0]
filtered_df["none"] = filtered_df.isnull().sum(axis=1)
filtered_df = filtered_df.sort_values(by="none", ascending=True).drop_duplicates(['gnps_libid']).sort_values(by="clinical_phase", ascending=False)
filtered_df['gnps_libid'] = [ids.split(", ") for ids in filtered_df['gnps_libid']]
filtered_df = filtered_df.explode(column='gnps_libid')
filtered_df.set_index('gnps_libid', inplace=True)
# filtered_df[filtered_df.duplicated(['gnps_libid'], keep=False)]
filtered_df

In [ ]:
filtered_df = pd.read_csv(df_file, sep="\t")
filtered_df[filtered_df["clinical_phase"]==4].groupby("inchi_key").count()
# filtered_df[filtered_df["drugcentral_administration_number"]==4]

In [ ]:
filtered_df[filtered_df["clinical_phase"]>0]

In [ ]:
df_file_mce = r"C:\git\msn_library\data\final_tables\mce_library_all_phase.tsv"
df = pd.read_csv(df_file_mce, sep="\t")

In [ ]:
s = {}
for inchikey, clinical in zip(df["inchi_key"], df["clinical_phase"]):
    s[inchikey] = clinical
for inchikey, clinical in zip(filtered_df["inchi_key"], filtered_df["clinical_phase"]):
    s[inchikey] = clinical
s

In [ ]:
sum([value==4 for key, value in s.items()])

In [ ]:
3420/2275

In [ ]:
filtered_df.loc["CCMSLIB00006119686", "canonical_smiles"]

In [ ]:
spectra = []
columns = ["compound_name", "canonical_smiles", "isomeric_smiles", "inchi", "inchi_key", "synonyms", "pubchem_cid", "iupac", "formula",
           "unii", "chembl_id", "drugbank_id", "drugcentral_id", "exact_mass", "clinical_phase",
           "drugcentral_administration", "drugbank_approved"]

with pyteomics.mgf.MGF(infile) as f_in:
    for spectrum_dict in tqdm(f_in):
        libid = spectrum_dict["params"]["spectrumid"]
        if libid in filtered_df.index:
            for col in columns:
                value = filtered_df.loc[libid, col]
                spectrum_dict["params"][col] = value if notnull(value) else ""

            spectra.append(spectrum_dict)

pyteomics.mgf.write(spectra, r'C:\git\msn_library\data\final_tables\gnps_drugs_clinical_phase_greater0.mgf')